In [73]:
import json
from datetime import datetime
import time
from threading import Thread,Lock

In [82]:
#Inizializzo id number
id_num=1

In [75]:
#Definisco semaforo
mutex=Lock()

In [76]:
#Assegna sezione censo
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

with open('/Volumes/HDD/Università/Tirocinio/Dati censo/sezioni_censimento_2011.geojson') as file:
    gj_sezioni = json.load(file)

In [77]:
features_sezioni=gj_sezioni['features']

In [78]:
def assegna_censo(coord_X,coord_Y):
    point=Point(coord_X,coord_Y)
    for f in features_sezioni:
        polygon = Polygon(f['geometry']['coordinates'][0])
        if polygon.contains(point):
            return f['properties']['SEZ']

_________ 

In [79]:
#Importo il file POI_lavoro.geojson
with open('/Volumes/HDD/Università/Tirocinio/POI/POI_lavoro.geojson') as file:
    gj_lavoro = json.load(file)

In [80]:
features=gj_lavoro['features']

In [83]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_lavoro=[]

#Definisco Thread
def setup_record_lavoro(f):
    global mutex
    global id_num
    global POI_lavoro
    POI={}
    
    #Estraggo il nome
    try:
        POI['nome']=f['properties']['name']
    except:
        try:
            POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
        except:
            POI['nome']=''

    #Estraggo tipologia
    try:    
        POI['tipo_edificio']=f['properties']['building']
    except:
        try:
            POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('building')
        except:
            POI['tipo_edificio']=''

    #Ripopolo campo 'Nome' se vuoto

    if POI['nome']=='':
        POI['nome']=POI['tipo_edificio'].capitalize()

    #Estraggo livelli edificio 

    try:    
        POI['lvl_edificio']=f['properties']['building:levels']
    except:
        POI['lvl_edificio']='1'

    #Stima numero dipendenti
    if POI['tipo_edificio']=='commercial':
        POI['dipendenti']=15*int(POI['lvl_edificio'])
    else:
        POI['dipendenti']=10*int(POI['lvl_edificio'])

    #Estraggo coordinate
    if f['geometry']['type']=='Point':
        coord_X=f['geometry']['coordinates'][0]
        coord_Y=f['geometry']['coordinates'][1]       
    else:
        if f['geometry']['type']=='LineString':
            coord_X=f['geometry']['coordinates'][0][0]
            coord_Y=f['geometry']['coordinates'][0][1]
        else:
            coord_X=f['geometry']['coordinates'][0][0][0]
            coord_Y=f['geometry']['coordinates'][0][0][1]
    
    #Correggo errore dati - coordinata come lista
    if type(coord_X) is list:
        POI['coord_X']=coord_X[0]
        POI['coord_Y']=coord_X[1]
    else:
        POI['coord_X']=coord_X
        POI['coord_Y']=coord_Y
    
    #Assegno zona censo
    POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])

    #Salvo il dict
    if POI['sezione_censo'] is not None:

        mutex.acquire()

        #Assegno ID
        POI['id']=id_num
        id_num=id_num+1

        POI_lavoro.append(POI)

        #Aggiungo record
        mutex.release()

#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for f in features:
    t=Thread(target=setup_record_lavoro, args=(f,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()
    
print('POI_lavoro creato - Numero POI: '+str(len(POI_lavoro)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-02-08 16:28:20
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_lavoro creato - Numero POI: 1837
Fine = 2021-02-08 16:30:22


_____________________________

In [84]:
#Importo il file POI_ristoro.geojson
with open('/Volumes/HDD/Università/Tirocinio/POI/POI_ristoro.geojson') as file:
    gj_ristoro = json.load(file)

In [85]:
features=gj_ristoro['features']

In [86]:
#Funzione assegna dipendenti ristoro
def num_dip_ristoro(tipo):
    switcher={
            'bar':3,
            'restaurant':10,
            'pub':5,
            'fast_food':7
         }
    return switcher.get(tipo,"errore")

In [87]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_ristoro=[]

#Estrazione dati necessari da POI_ristoro.geojson
def setup_record_ristoro(f):
    POI={}
    
    global POI_ristoro
    global id_num
    global mutex
    
    #Estraggo il nome
    try:
        POI['nome']=f['properties']['name']
    except:
        try:
            POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
        except:
            POI['nome']=''
            
    #Estraggo tipologia
    try:    
        POI['tipo_edificio']=f['properties']['amenity']
    except:
        try:
            POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('amenity')
        except:
            POI['tipo_edificio']=''
    
    #Ripopolo campo 'Nome' se vuoto
    if POI['nome']=='':
        POI['nome']=POI['tipo_edificio'].capitalize()
    
    #Stima numero dipendenti
    POI['dipendenti']=num_dip_ristoro(POI['tipo_edificio'])
    
    #Estraggo coordinate
    if f['geometry']['type']=='Point':
        coord_X=f['geometry']['coordinates'][0]
        coord_Y=f['geometry']['coordinates'][1]       
    else:
        if f['geometry']['type']=='LineString':
            coord_X=f['geometry']['coordinates'][0][0]
            coord_Y=f['geometry']['coordinates'][0][1]
        else:
            coord_X=f['geometry']['coordinates'][0][0][0]
            coord_Y=f['geometry']['coordinates'][0][0][1]
            
    POI['coord_X']=coord_X
    POI['coord_Y']=coord_Y 

    #Assegno zona censo
    POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])
    
    #Salvo il dict
    if POI['sezione_censo'] is not None:

        mutex.acquire()

        #Assegno ID
        POI['id']=id_num
        id_num=id_num+1

        POI_ristoro.append(POI)

        #Aggiungo record
        mutex.release()
        
#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for f in features:
    t=Thread(target=setup_record_ristoro, args=(f,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()

print('POI_ristoro aggiornato - Numero POI: '+str(len(POI_ristoro)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-02-08 16:31:14
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_ristoro aggiornato - Numero POI: 2984
Fine = 2021-02-08 16:33:45


________

In [88]:
#Importo il file POI_turismo.geojson
with open('/Volumes/HDD/Università/Tirocinio/POI/POI_turismo.geojson') as file:
    gj_turismo = json.load(file)

In [89]:
features=gj_turismo['features']

In [90]:
#Funzione assegna dipendenti turismo
def num_dip_turismo(tipo):
    switcher={
            'artwork':2,
            'attraction':6,
            'museum':10,
            'park':5,
            'monument':3
         }
    return switcher.get(tipo,"errore")

In [91]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_turismo=[]

#Estrazione dati necessari da POI_turismo.geojson
def setup_record_turismo(f):
    POI={}
    
    global id_num
    global mutex
    global POI_turismo
    
    #Estraggo il nome
    try:
        POI['nome']=f['properties']['name']
    except:
        try:
            POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
        except:
            POI['nome']=''
            
    #Estraggo tipologia
    try:    
        POI['tipo_edificio']=f['properties']['tourism']
    except:
        try:
            POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('leisure')
        except:
            try:
                POI['tipo_edificio']=f['properties']['historic']
            except:
                try:
                    POI['tipo_edificio']=f['properties']['leisure']
                except:
                    POI['tipo_edificio']=''
    
    #Ripopolo campo 'Nome' se vuoto
    if POI['nome']=='':
        POI['nome']=POI['tipo_edificio'].capitalize()
    
    #Stima numero dipendenti
    POI['dipendenti']=num_dip_turismo(POI['tipo_edificio'])
    
    #Estraggo coordinate
    if f['geometry']['type']=='Point':
        coord_X=f['geometry']['coordinates'][0]
        coord_Y=f['geometry']['coordinates'][1]       
    else:
        if f['geometry']['type']=='LineString':
            coord_X=f['geometry']['coordinates'][0][0]
            coord_Y=f['geometry']['coordinates'][0][1]
        else:
            coord_X=f['geometry']['coordinates'][0][0][0]
            coord_Y=f['geometry']['coordinates'][0][0][1]
            
    #Correggo errore dati - coordinata come lista
    if type(coord_X) is list:
        POI['coord_X']=coord_X[0]
        POI['coord_Y']=coord_X[1]
    else:
        POI['coord_X']=coord_X
        POI['coord_Y']=coord_Y    
    
    #Assegno zona censo
    POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])
    
    #Salvo il dict
    if POI['sezione_censo'] is not None:

        mutex.acquire()

        #Assegno ID
        POI['id']=id_num
        id_num=id_num+1

        POI_turismo.append(POI)

        #Aggiungo record
        mutex.release()
        
#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for f in features:
    t=Thread(target=setup_record_turismo, args=(f,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()

print('POI_turismo aggiornato - Numero POI: '+str(len(POI_turismo)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-02-08 16:34:11
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_turismo aggiornato - Numero POI: 741
Fine = 2021-02-08 16:35:00


_______

In [92]:
#Importo il file POI_spese.geojson
with open('/Volumes/HDD/Università/Tirocinio/POI/POI_spese.geojson') as file:
    gj_spese = json.load(file)

In [93]:
features=gj_spese['features']

In [94]:
#Funzione assegna dipendenti spese
def num_dip_spese(tipo):
    switcher={
            'clothes':6,
            'pharmacy':4,
            'supermarket':10,
            'tailor':4,
            'tobacco':3
         }
    return switcher.get(tipo,"errore")

In [95]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_spese=[]

#Estrazione dati necessari da POI_spese.geojson
def setup_record_spese(f):
    POI={}
    
    global id_num
    global mutex
    global POI_spese
    
    #Estraggo il nome
    try:
        POI['nome']=f['properties']['name']
    except:
        try:
            POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
        except:
            POI['nome']=''
            
    #Estraggo tipologia
    try:    
        POI['tipo_edificio']=f['properties']['shop']
    except:
        try:
            POI['tipo_edificio']=f['properties']['amenity']
        except:
            POI['tipo_edificio']=''

    #Ripopolo campo 'Nome' se vuoto
    if POI['nome']=='':
        POI['nome']=POI['tipo_edificio'].capitalize()
    
    #Stima numero dipendenti
    POI['dipendenti']=num_dip_spese(POI['tipo_edificio'])
    
    #Estraggo coordinate
    if f['geometry']['type']=='Point':
        coord_X=f['geometry']['coordinates'][0]
        coord_Y=f['geometry']['coordinates'][1]       
    else:
        if f['geometry']['type']=='LineString':
            coord_X=f['geometry']['coordinates'][0][0]
            coord_Y=f['geometry']['coordinates'][0][1]
        else:
            coord_X=f['geometry']['coordinates'][0][0][0]
            coord_Y=f['geometry']['coordinates'][0][0][1]
            
    POI['coord_X']=coord_X
    POI['coord_Y']=coord_Y   
    
    #Assegno zona censo
    POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])
    
    #Salvo il dict
    if POI['sezione_censo'] is not None:

        mutex.acquire()

        #Assegno ID
        POI['id']=id_num
        id_num=id_num+1

        POI_spese.append(POI)

        #Aggiungo record
        mutex.release()
        
#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for f in features:
    t=Thread(target=setup_record_spese, args=(f,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()

print('POI_spese aggiornato - Numero POI: '+str(len(POI_spese)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-02-08 16:35:00
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_spese aggiornato - Numero POI: 1557
Fine = 2021-02-08 16:36:15


_______

In [96]:
#Importo il file POI_scuola.geojson
with open('/Volumes/HDD/Università/Tirocinio/POI/POI_scuola.geojson') as file:
    gj_scuola = json.load(file)

In [97]:
features=gj_scuola['features']

In [98]:
#Funzione assegna dipendenti scuola
def num_dip_scuola(tipo):
    switcher={
            'library':6,
            'school':20,
            'university':40
         }
    return switcher.get(tipo,"errore")

In [99]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#Creazione struttura
POI_scuola=[]

#Estrazione dati necessari da POI_spese.geojson
def setup_record_scuola(f):
    POI={}
    
    global id_num
    global mutex
    global POI_scuola
    
    #Estraggo il nome
    try:
        POI['nome']=f['properties']['name']
    except:
        try:
            POI['nome']=f['properties']['@relations'][0]['reltags'].get('name')
        except:
            POI['nome']=''
            
    #Estraggo tipologia
    try:    
        POI['tipo_edificio']=f['properties']['amenity']
    except:
        POI['tipo_edificio']=''
    
    #Ripopolo campo 'Nome' se vuoto
    if POI['nome']=='':
        POI['nome']=POI['tipo_edificio'].capitalize()
    
    #Stima numero dipendenti
    POI['dipendenti']=num_dip_scuola(POI['tipo_edificio'])
    
    #Estraggo coordinate
    if f['geometry']['type']=='Point':
        coord_X=f['geometry']['coordinates'][0]
        coord_Y=f['geometry']['coordinates'][1]       
    else:
        if f['geometry']['type']=='LineString':
            coord_X=f['geometry']['coordinates'][0][0]
            coord_Y=f['geometry']['coordinates'][0][1]
        else:
            coord_X=f['geometry']['coordinates'][0][0][0]
            coord_Y=f['geometry']['coordinates'][0][0][1]
            
    POI['coord_X']=coord_X
    POI['coord_Y']=coord_Y     
    
    #Assegno zona censo
    POI['sezione_censo']=assegna_censo(POI['coord_X'],POI['coord_Y'])
    
    #Salvo il dict
    if POI['sezione_censo'] is not None:

        mutex.acquire()

        #Assegno ID
        POI['id']=id_num
        id_num=id_num+1

        POI_scuola.append(POI)

        #Aggiungo record
        mutex.release()
        
#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for f in features:
    t=Thread(target=setup_record_scuola, args=(f,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()


print('POI_scuola aggiornato - Numero POI: '+str(len(POI_scuola)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-02-08 16:36:15
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
POI_scuola aggiornato - Numero POI: 77
Fine = 2021-02-08 16:36:17


________

In [98]:
#Trasformo dict to json dump
with open('./json_posti_lavoro.json', 'w') as fp:
    POI_posti_lavoro=[]
    for r in POI_lavoro:
        POI_posti_lavoro.append(r)
    for r in POI_ristoro:
        POI_posti_lavoro.append(r)
    for r in POI_turismo:
        POI_posti_lavoro.append(r)
    for r in POI_scuola:
        POI_posti_lavoro.append(r)
    for r in POI_spese:
        POI_posti_lavoro.append(r)
        
    json.dump(POI_posti_lavoro, fp)

___________

In [99]:
#Creo json con capienza per ogni tipologia
for p in POI_ristoro:
    del p['dipendenti']

for p in POI_scuola:
    del p['dipendenti']

for p in POI_spese:
    del p['dipendenti']

for p in POI_turismo:
    del p['dipendenti']

In [100]:
#Funzione assegna capienza ristoro
def capienza_ristoro(tipo):
    switcher={
            'bar':20,
            'restaurant':40,
            'pub':30,
            'fast_food':35
         }
    return switcher.get(tipo,"errore")

In [101]:
#Funzione assegna capienza turismo
def capienza_turismo(tipo):
    switcher={
            'artwork':20,
            'attraction':50,
            'museum':50,
            'park':150,
            'monument':40
         }
    return switcher.get(tipo,"errore")

In [102]:
#Funzione assegna capienza spese
def capienza_spese(tipo):
    switcher={
            'clothes':30,
            'pharmacy':10,
            'supermarket':50,
            'tailor':15,
            'tobacco':10
         }
    return switcher.get(tipo,"errore")

In [103]:
#Funzione assegna capienza scuola
def capienza_scuola(tipo):
    switcher={
            'library':60,
            'school':200,
            'university':350
         }
    return switcher.get(tipo,"errore")

In [104]:
#Aggiorno POI
for p in POI_ristoro:
    p['capienza']=capienza_ristoro(p['tipo_edificio'])

for p in POI_scuola:
    p['capienza']=capienza_scuola(p['tipo_edificio'])

for p in POI_spese:
    p['capienza']=capienza_spese(p['tipo_edificio'])

for p in POI_turismo:
    p['capienza']=capienza_turismo(p['tipo_edificio'])

In [105]:
#Salvo singoli dump json
with open('./json_ristoro.json', 'w') as fp:
    json.dump(POI_ristoro,fp)
with open('./json_turismo.json', 'w') as fp:
    json.dump(POI_turismo,fp)
with open('./json_scuola.json', 'w') as fp:
    json.dump(POI_scuola,fp)
with open('./json_spese.json', 'w') as fp:
    json.dump(POI_spese,fp)

_______________

In [31]:
#Pulizia POI Residenze -- DA Rimuovere
#Importo il file POI_residenze.geojson
with open('/Volumes/HDD/Università/Tirocinio/POI/POI_residenze.geojson') as file:
    gj_residenze = json.load(file)

In [32]:
features=gj_residenze['features']

In [33]:
'''
#Funzione assegna capienza residenze
import random

def capienza_residenze(tipo,lvl):
    if tipo == 'apartments':
        capienza=0
        for i in range(0,lvl):
            capienza=capienza+random.randint(1,5)
        return capienza
    if tipo == 'residential':
        capienza=0
        for i in range(0,lvl):
            capienza=capienza+random.randint(2,8)
        return capienza
'''

In [45]:
id_num=1
tipo=[]

#Creazione struttura
POI_residenze=[]

#Estrazione dati necessari da POI_residenze.geojson
for f in features:
    POI={}
    
    #Assegno ID
    POI['id']=id_num
    id_num=id_num+1
            
    #Estraggo tipologia
    try:    
        POI['tipo_edificio']=f['properties']['building']
    except:
        try:
            POI['tipo_edificio']=f['properties']['@relations'][0]['reltags'].get('building')
        except:
            POI['tipo_edificio']=''

    #Ripopolo campo 'Nome' se vuoto
    POI['nome']=POI['tipo_edificio'].capitalize()
        
    #Estraggo livelli edificio 
    try:    
        POI['lvl_edificio']=f['properties']['building:levels']
    except:
        POI['lvl_edificio']='1'
        
    #Assegno capienza
    #POI['num_residenti']=capienza_residenze(POI['tipo_edificio'],int(POI['lvl_edificio']))
    
    #Estraggo coordinate
    if f['geometry']['type']=='Point':
        coord_X=f['geometry']['coordinates'][0]
        coord_Y=f['geometry']['coordinates'][1]       
    else:
        if f['geometry']['type']=='LineString':
            coord_X=f['geometry']['coordinates'][0][0]
            coord_Y=f['geometry']['coordinates'][0][1]
        else:
            coord_X=f['geometry']['coordinates'][0][0][0]
            coord_Y=f['geometry']['coordinates'][0][0][1]
            
    POI['coord_X']=coord_X
    POI['coord_Y']=coord_Y  
    
    #Salvo il dict
    if type(coord_X) is not list:
        POI_residenze.append(POI)

print('POI_residenze aggiornato - Numero POI: '+str(len(POI_residenze)))


POI_residenze aggiornato - Numero POI: 28817


In [46]:
with open('./json_residenze.json', 'w') as fp:
    json.dump(POI_residenze, fp)